<a href="https://colab.research.google.com/github/aakarshhh/AI_ML/blob/main/PyTorch_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#PyTorch Tutorial (Beginner)

## Prerequisites:
This tutorial assumes that you already have atleast beginner level of experience with the things below. 
 - Python ([freeCodeCamp Crash Course](https://www.youtube.com/watch?v=LHBE6Q9XlzI), [Videos by Sentdex](https://www.youtube.com/playlist?list=PLQVvvaa0QuDeAams7fkdcwOGBpGdHpXln))
 - NumPy ([freeCodeCamp Crash Course](https://www.freecodecamp.org/news/numpy-crash-course-build-powerful-n-d-arrays-with-numpy/) & its corresponding [colab notebook](https://colab.research.google.com/drive/1Oa8J_sZXACQJEiMqANIHkftMgUrqSpVt#scrollTo=ITrCTnT6RkWP))



## Installation:
Throughout this course, we wont be doing any heavy computations, so its better to stick to Google Colab as every possible library you will need will come pre-installed. 

If you still wish to try and install it locally, kindly refer to the links below that have some of best the explanations for beginners:
- [Pytorch (Official)](https://pytorch.org/get-started/locally/)
- [Article by JournalDev (CPU only, Linux/Windows/MacOS)](https://www.journaldev.com/35965/pytorch-installation)
- [Article by DeepLizard (A bit old)](https://deeplizard.com/learn/video/UWlFM0R_x6I)

Again, if the instructions in the above links look like gibbersh, we strongly recommend you to stick with Google Colab atleast until you get familarized with it.

## Contents of this tutorial
- Parallels between PyTorch and NumPy
- How to train a model?
- Things to keep in mind


# Parallels between PyTorch and NumPy


In [ ]:
import numpy as np

print("NumPy")
print("===========")
print(np.mean([1.0, 3.0]))
print(np.sum([1.0, 3.0]))
print(np.exp([1.0, 3.0]))

NumPy
2.0
4.0
[ 2.71828183 20.08553692]


In [ ]:
import torch

print("PyTorch")
print("===========")
print(torch.mean(torch.tensor([1.0, 3.0])))
print(torch.sum(torch.tensor([1.0, 3.0])))
print(torch.exp(torch.tensor([1.0, 3.0])))

PyTorch
tensor(2.)
tensor(4.)
tensor([ 2.7183, 20.0855])


In [ ]:
print("NumPy")
print("===========")
x_np = np.array([1.0,3.0])
print(x_np.mean())
print(x_np.sum())

print("\nPyTorch")
print("===========")
x_pt = torch.tensor([1.0,3.0])
print(x_pt.mean())
print(x_pt.sum())

NumPy
2.0
4.0

PyTorch
tensor(2.)
tensor(4.)


In [ ]:
print("NumPy")
print("===========")
print(np.random.rand(5))

print("\nPyTorch")
print("===========")
print(torch.rand(5))

NumPy
[0.82565812 0.60084822 0.42939775 0.32658846 0.82588313]

PyTorch
tensor([0.5673, 0.3558, 0.5948, 0.4870, 0.2285])


# Numpy Bridge

In [ ]:
narr = np.arange(0, 9).reshape(3, 3).astype(np.float)

In [ ]:
narr

array([[0., 1., 2.],
       [3., 4., 5.],
       [6., 7., 8.]])

In [ ]:
t1 = torch.from_numpy(narr)

In [ ]:
t1

tensor([[0., 1., 2.],
        [3., 4., 5.],
        [6., 7., 8.]], dtype=torch.float64)

In [ ]:
t2 = t1.view(1, 9)

In [ ]:
t2

tensor([[0., 1., 2., 3., 4., 5., 6., 7., 8.]], dtype=torch.float64)

In [ ]:
narr *= 2
narr

array([[ 0.,  2.,  4.],
       [ 6.,  8., 10.],
       [12., 14., 16.]])

In [ ]:
t1

tensor([[ 0.,  2.,  4.],
        [ 6.,  8., 10.],
        [12., 14., 16.]], dtype=torch.float64)

In [ ]:
t2

tensor([[ 0.,  2.,  4.,  6.,  8., 10., 12., 14., 16.]], dtype=torch.float64)

In [ ]:
t1.storage()

 0.0
 2.0
 4.0
 6.0
 8.0
 10.0
 12.0
 14.0
 16.0
[torch.DoubleStorage of size 9]

In [ ]:
t1.storage().data_ptr()

94084183236224

In [ ]:
t2.storage().data_ptr()

94084183236224

If we can do everything in numpy too, then why do we even need PyTorch?

- PyTorch can do everything on GPU too. (We will see this in detail towards the end. It is just a one step process)
- PyTorch takes care of most of the things required for Deep Learning like back propagation, dataset handling, etc. in the background without much user intervention.

# How to train a model?

Lets train and evaluate a classifier on [CIFAR-10](https://www.cs.toronto.edu/~kriz/cifar.html) dataset. 

First we will go over the steps very roughly. Don't try to understand every single but try and absorb the overall idea of what we are trying to do. Then a step by step explanation will be provided on what is happening.
- Dataset Preparation
- Define some Neural Network for training
- Training Loop
  - Forward Propagation
  - Loss
  - Backward Propagation
  - Train and test accuracy
- Save/Load a trained model
- How to train on GPU?


## Rough Overview

In [ ]:
# Imports
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# Dataset Preparation
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

# Define some Neural Network for training
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

# Training Loop
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data

        optimizer.zero_grad()

        # Forward Propagation
        outputs = net(inputs)

        # Loss
        loss = criterion(outputs, labels)

        # Backward Propagation
        loss.backward()
        
        if i%2000 == 0:
          print(f"[{epoch}, {i}] Loss: {loss.item()}")
        
        optimizer.step()

print('Finished Training')

# Train Accuracy
correct = 0
total = 0
with torch.no_grad():
    for data in trainloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(f"Train Accuracy: {100*(correct/total)}")

# Test Accuracy
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(f"Test Accuracy: {100*(correct/total)}")

torch.save(net, "trained_model.pt")
print("Model saved!")

trained_network = torch.load("trained_model.pt")
print("Model Loaded!")

That must have taken alot of time to train. So, before we get into the step by step explanation, lets see what to change in the above code to **run everything on GPU**!

- Click on **Runtime** -> **Change runtume type** -> **Hardward accelerator** -> **GPU**
- Change `net = Net()` to `net = Net().cuda()`
- Below `inputs, labels = data` in the training loop add another line `inputs, labels = inputs.cuda(), labels.cuda()`.
- Below `images, labels = data` in the train/test accuracy step add another line `inputs, labels = inputs.cuda(), labels.cuda()`.


## Step by step explanation
Now that we have an overall idea of what is happening, lets try to understand everything one step at a time.


### Imports

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

### Dataset Preparation

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


### Define some Neural Network for training

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net().cuda()

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

### Training Loop

In [ ]:
for epoch in range(2):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0): # loop over the mini-batches of data 
        inputs, labels = data
        inputs, labels = inputs.cuda(), labels.cuda()

        optimizer.zero_grad()

        # Forward Propagation
        outputs = net(inputs)

        # Loss
        loss = criterion(outputs, labels)

        # Backward Propagation
        loss.backward()
        if i%2000 == 0:
          print(f"[{epoch}, {i}] Loss: {loss.item()}")
        
        optimizer.step()

print('Finished Training')

[0, 0] Loss: 2.330172300338745
[0, 2000] Loss: 2.200355052947998
[0, 4000] Loss: 1.5364435911178589
[0, 6000] Loss: 1.9352993965148926
[0, 8000] Loss: 2.05306077003479
[0, 10000] Loss: 0.6619178056716919
[0, 12000] Loss: 2.002880573272705
[1, 0] Loss: 2.3936924934387207
[1, 2000] Loss: 0.9825661182403564
[1, 4000] Loss: 1.2533040046691895
[1, 6000] Loss: 0.5188676118850708
[1, 8000] Loss: 1.3815442323684692
[1, 10000] Loss: 1.061977505683899
[1, 12000] Loss: 1.1666178703308105
Finished Training


### Train/Test Accuracy

In [ ]:
# Train Accuracy
correct = 0
total = 0
with torch.no_grad():
    for data in trainloader:
        images, labels = data
        images, labels = images.cuda(), labels.cuda()
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(f"Train Accuracy: {100*(correct/total)}")

# Test Accuracy
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.cuda(), labels.cuda()
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(f"Test Accuracy: {100*(correct/total)}")

Train Accuracy: 57.084
Test Accuracy: 55.81


### Save/Load a trained model

In [ ]:
torch.save(net, "trained_model.pt")
print("Model saved!")

trained_network = torch.load("trained_model.pt")
print("Model Loaded!")

Model saved!
Model Loaded!


# Things to keep in mind
- Boiler Plate Code - https://github.com/victoresque/pytorch-template
- PyTorch Community - https://discuss.pytorch.org/
- Reproducibility Challenge - https://paperswithcode.com/rc2020
- Nice article: [A Recipe for Training Neural Networks by Andrej Karpathy](http://karpathy.github.io/2019/04/25/recipe/)

